In [1]:
using Pkg
Pkg.activate(String(@__DIR__) * "/..")
Pkg.instantiate()

      From worker 5:	 Activating environment at `c:\Users\nodak\Documents\Nicolas Descouens\Thesis\Conduction\Project.toml`
      From worker 2:	 Activating environment at `c:\Users\nodak\Documents\Nicolas Descouens\Thesis\Conduction\Project.toml`

 Activating environment at `c:\Users\nodak\Documents\Nicolas Descouens\Thesis\Conduction\Project.toml`



      From worker 4:	 Activating environment at `c:\Users\nodak\Documents\Nicolas Descouens\Thesis\Conduction\Project.toml`
      From worker 3:	 Activating environment at `c:\Users\nodak\Documents\Nicolas Descouens\Thesis\Conduction\Project.toml`
[2, 3, 4, 5]


In [2]:
@everywhere begin
    include("../src/Conduction.jl");
    using Plots, VegaLite, LaTeXStrings, QuadGK;
end

In [3]:
@everywhere begin
    semiconductor = Conduction.Semiconductor(
    1.38 * 10^-23, # Boltzman constant (J.K^-1)
    1.6*10^-19, # Electron's charge (C)
    10^7, # decay constant of the assumed hydrogen-like localized state wave functions (cm^-1)
    0.1 * 1.6*10^-19, # Mode effect of the phonons (J)
    2.1 * 10^18, # intrinsic semiconductor's density (cm^-3)
    0, # Doping states' density (cm^-3)
    0.1 * 1.6 * 10^-19, # Energy to a vacant target site (J)
    -2*10^3, # Field (V.cm^-1)
    10^13, # Base electron jump rate
    -10.0, # Fermi level (J)
    2.3, # Intrinsic semiconductor's gaussian width (J)
    2.7, # Doping states' gaussian width (J)
    2.7, # Amount of disorder
    1 # Phonon lower frequency
    );

    T = 300;
end

In [4]:
semiconductor.Uf(T) / semiconductor.SigmaI(T)

-4.3478260869565215

Compared to papers, the ratio seems too high

In [5]:
Conduction.overallEin(semiconductor, Conduction.RnnVRH, T, 15) / semiconductor.k * T / semiconductor.q

3.2951394948128266e6

In [6]:
println(semiconductor.Uf(T) / (1.6 * 10^-19))
println(semiconductor.SigmaI(T) / (1.6 * 10^-19))

-0.25875
0.059512499999999996


Fermi level is too high.

 We'll find $E_F$ in such way that $\dfrac{E_F}{\sigma} = -6.7$

In [7]:
6.7 * semiconductor.SigmaI(T) / semiconductor.k / T

15.41

In [8]:
semiconductor.Uf(T) = -15.41 * semiconductor.k * T;
Conduction.overallEin(semiconductor, Conduction.RnnVRH, T, 15) * semiconductor.k * T / semiconductor.q

3.3109355258407835e6

We'll now take $E_F$ in such way that $\dfrac{E_F}{\sigma} = 6.2$ and $\sigma = 3 k_BT$

In [9]:
semiconductor.SigmaI(T) = 3 * semiconductor.k * T
6.2 * semiconductor.SigmaI(T) / semiconductor.k / T

18.6

In [10]:
semiconductor.Uf(T) = -18.6 * semiconductor.k * T;
Conduction.overallEin(semiconductor, Conduction.RnnVRH, T, 15) / semiconductor.k * T / semiconductor.q

3.495287003353969e6

We'll take $E_F = -15.41$ and keep the same $\sigma$ value.

In [11]:
semiconductor.Uf(T) = -15.41 * semiconductor.k * T;
Conduction.overallEin(semiconductor, Conduction.RnnVRH, T, 15) / semiconductor.k * T / semiconductor.q

3.490489152587037e6

$E_F$ does not seem to have a great influence on the einstein ratio.